In [1]:
from typing import Dict, Tuple, List
from math import sqrt
from dataclasses import dataclass
from gurobipy import Model, GRB, tupledict
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

In [4]:
import numpy as np
import os
import re

class TspInstance:
    """ An instance of the Travelling Salesman Problem. """

    def __init__(self, n_vertices, cost, x=None, y=None):
        """
        Initialize a TSP instance.

        Parameters
        ----------
        n_vertices : int
            Number of vertices in the graph.
        cost : dict[tuple[int, int], float]
            Dictionary with arcs as key, and their corresponding costs as values.
        x : list[float], optional
            X coordinates of the vertices (for plotting).
        y : list[float], optional
            Y coordinates of the vertices (for plotting).
        """
        self.n_vertices = n_vertices
        self.cost = cost
        self.x = x if x is not None else []
        self.y = y if y is not None else []

# You can now load your TSP data and create an instance of this class:

# Load data
DIR = 'data'
problem = 'rbg323.atsp'  # Example problem
file_name = os.path.join(DIR, problem)

# Assume `content` is the content of the file read in
# Implement your file reading logic here
# ...

numeric_part = re.search(r'\d+', problem).group()
n = int(numeric_part)

if n == 64:
    n = 65
    
with open(file_name, 'r') as file:
    content = file.read()
    
data_lines = content.split('\n')
start_index = data_lines.index('EDGE_WEIGHT_SECTION')
edge_weight_lines = data_lines[start_index + 1:][:-2]

def flatten(l):
    return [item for sublist in l for item in sublist]

distance_matrix = [list(map(int, line.split())) for line in edge_weight_lines if line.strip()]
distance_list = flatten(distance_matrix)

my_matrix = np.array(distance_list).reshape((n, n))
V = range(n)

cost = {(i, j): my_matrix[i, j] for i in V for j in V if i != j}

# Creating an instance of TspInstance
tsp_instance = TspInstance(n_vertices=n, cost=cost)



FileNotFoundError: [Errno 2] No such file or directory: 'data/rbg323.atsp'

In [6]:
@dataclass
class TspSolution:
    """ Solution to an instance of the Travelling Salesman Problem.

        Attributes
        ----------
        instance: TspInstance
            Reference to the TSP instance being solved.
        tour: List[int]
            Ordered list of vertices visited (without repeating the first one).
        cost: float
            Travel cost of the tour.
    """

    instance: TspInstance
    tour: List[int]
    cost: float

    def plot(self) -> Tuple[Figure, plt.Axes]:
        """ Plots the solution of the TSP.
        
            Vertices are black dots, the solution is represented by red lines.

            Returns
            -------
            A tuple with the figure and axes object from matplotlib.    
        """

        xy = [(self.instance.x[i], self.instance.y[i]) for i in self.tour + [self.tour[0]]]

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.scatter(self.instance.x, self.instance.y, color='black')
        
        for pt1, pt2 in zip(xy[1:], xy[:-1]):
            x1, y1 = pt1
            x2, y2 = pt2
            ax.plot([x1, x2], [y1, y2], color='red')

        ax.set_title(f"TSP Solution. Num vertices: {self.instance.n_vertices}. Tour cost: {self.cost:.2f}.")
        ax.set_axis_off()

        return fig, ax
    
class TspIntegerBCSolver:
    """ 
    Solver for the Travelling Salesman Problem using the branch-and-cut algorithm.

    This solver applies the Dantzig-Fulkerson-Johnson (DFJ) formulation of the TSP
    and focuses on separating subtour elimination constraints (SECs) only on integer solutions.

    Attributes:
    _instance : TspInstance
        The instance of the TSP to be solved.
    _V : List[int]
        List of vertex indices in the TSP instance.
    _A : List[Tuple[int, int]]
        List of arcs (edges between pairs of vertices) in the TSP instance.
    _model : Model
        The Gurobi model for the TSP instance.
    _x : tupledict
        Gurobi variables representing whether an arc is included in the tour.
    """

    def __init__(self, instance: TspInstance, cardinality=None):
        """
        Initializes the solver with a TSP instance and builds the model.

        Parameters:
        instance (TspInstance): The TSP instance to be solved.
        """
        # Store the TSP instance and create lists of vertices and arcs
        self._instance = instance
        self._V = list(range(self._instance.n_vertices))
        self._A = list(self._instance.cost.keys())
        self.lazy_constraints_added = []  # Initialize an empty list to store lazy constraints
        self.cardinality = cardinality  # Store the cardinality parameter

        # Build the optimization model
        self.__build_model()

    def __build_model(self):
        """ 
        Builds the initial DFJ model for the TSP without subtour elimination constraints (SECs).
        """
        # Initialize the Gurobi model
        self._model = Model()

        # Add variables representing whether each arc is included in the tour
        self._x = self._model.addVars(self._A, obj=self._instance.cost, vtype=GRB.BINARY, name='x')

        # Add constraints to ensure exactly one outgoing and one incoming arc for each vertex
        self._model.addConstrs((self._x.sum(i, '*') == 1 for i in self._V), name='outgoing')
        self._model.addConstrs((self._x.sum('*', i) == 1 for i in self._V), name='incoming')

        # Conditionally add cardinality constraints if the cardinality is set to 2
        if self.cardinality == 2 or self.cardinality == 3:
            self._model.addConstrs((self._x[i, j] + self._x[j, i] <= 1 for i in self._V for j in self._V if i != j), name='cardinality two')

        # Add constraints to ensure x(i,j) + x(j,k) +x(k,i) <= 2
        if self.cardinality == 3:
            self._model.addConstrs((self._x[i, j] + self._x[j, k] + self._x[k, i]<= 2 for i in self._V for j in self._V for k in self._V if i != j and j != k and k != i), name='cardinality three')


    def __next_vertex(self, i: int) -> int:
        """
        Finds the next vertex in the current solution, starting from vertex i.

        Parameters:
        i (int): The starting vertex index.

        Returns:
        int: The index of the next vertex in the tour.
        """
        # Ensure the vertex index is valid
        assert 0 <= i < self._instance.n_vertices

        # Loop through all vertices to find the next one in the tour
        for j in self._V:
            if i != j:
                try:
                    # Inside a callback, retrieve the solution using cbGetSolution
                    val = self._model.cbGetSolution(self._x[i, j])
                except:
                    # Otherwise, access the variable's value directly
                    val = self._x[i, j].X

                # If the arc is part of the solution, return the next vertex
                if val > 0.5:
                    return j

        # If no successor is found, raise an error
        assert False, f"Vertex {j} has no successor?!"

    def __tour_staring_at(self, i: int) -> List[int]:
        """
        Constructs a tour (or subtour) starting at vertex i based on the current solution.

        Parameters:
        i (int): The starting vertex index.

        Returns:
        List[int]: A list of vertex indices representing the tour.
        """
        # Ensure the vertex index is valid
        assert 0 <= i < self._instance.n_vertices

        # Initialize the tour with the starting vertex
        tour = [i]
        current = self.__next_vertex(i)

        # Continue adding vertices to the tour until it loops back to the start
        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __add_sec_for(self, S: List[int]) -> None:
        """
        Adds a subtour elimination constraint (SEC) for the vertices in set S.

        Parameters:
        S (List[int]): A list of vertex indices forming a subtour.
        """
        # Ensure the subtour has a valid number of vertices
        assert 0 < len(S) < self._instance.n_vertices
   
        # Record the constraint information
        constraint_info = {
            'subtour': S,
            'constraint': sum(self._x[i, j] for (i, j) in self._A if i in S and j not in S) >= 1
        }

        self.lazy_constraints_added.append(constraint_info)
        
        # Add the actual lazy constraint to the model
        self._model.cbLazy(sum(self._x[i, j] for (i, j) in self._A if i in S and j not in S) >= 1)

    def __separate(self, where: int) -> None:
        """ Separates eventual violated SECs. """

        if where != GRB.Callback.MIPSOL:
            return
        
        # Set of vertices not yet placed in a tour
        remaining = set(self._V)

        while len(remaining) > 0:
            # Get any vertex in set `remaining`
            current = next(iter(remaining))

            # Get the subtour which includes the vertex
            subtour = self.__tour_staring_at(current)

            # If the tour visits all vertices, nothing to do
            if len(subtour) == self._instance.n_vertices:
                return

            # Otherwise, it's a subtour => Add a SEC
            self.__add_sec_for(subtour)

            # Update set `remaining`
            remaining -= set(subtour)

    def solve(self) -> TspSolution:
        """ Solves the TSP DFJ model and returns the solution.
        
            It throws a RuntimeError if Gurobi cannot solve the model to optimality.

            Returns
            -------
            A TspSolution object with details about the solution.
        """

        # We must set this parameter to use callbacks
        self._model.setParam(GRB.Param.LazyConstraints, 1)

        # Gurobi always passes two parameters to the callback: `model` and `where`
        self._model.optimize(lambda _, where: self.__separate(where))

        if self._model.Status != GRB.OPTIMAL:
            raise RuntimeError(f"Could not find the optimal solution. Gurobi status = {self._model.Status}")

        return TspSolution(
            instance=self._instance,
            tour=self.__tour_staring_at(0),
            cost=self._model.ObjVal
        )


## Baseline

In [4]:
i = TspInstance(file='tsp-instance.txt')
s = TspIntegerBCSolver(instance=i, cardinality=None)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-12


In [5]:
# Check how many times cardinality constraints where violated
for entry in s.lazy_constraints_added:
    subtour = entry['subtour']
    constraint = entry['constraint']
    
    # Check if the subtour length is exactly 2
    if len(subtour) == 2:
        # The subtour contains exactly two vertices i and j
        i, j = subtour
        # The corresponding constraint is for i to j and j to i
        print(f"Lazy constraint for subtour {subtour}")


In [6]:
# Check how many times cardinality constraints where violated
for entry in s.lazy_constraints_added:
    subtour = entry['subtour']
    constraint = entry['constraint']
    

    # Check if the subtour length is exactly 2
    if len(subtour) == 3:
        # The subtour contains exactly two vertices i and j
        i, j, k = subtour
        # The corresponding constraint is for i to j and j to i
        print(f"Lazy constraint for subtour {subtour}")

In [7]:
solution = s.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 2450 columns and 4900 nonzeros
Model fingerprint: 0x42c18cfd
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2543.7515673
Presolve time: 0.01s
Presolved: 100 rows, 2450 columns, 4900 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 4.552846e+02, 81 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  455.28464    0    - 2543.75157  455.28464  82.1%     -    0

## Cardinality 2

In [8]:
s_2 = TspIntegerBCSolver(instance=i, cardinality=2)
solution = s_2.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2550 rows, 2450 columns and 9800 nonzeros
Model fingerprint: 0x89ddbe4d
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2543.7515673
Presolve removed 1225 rows and 0 columns
Presolve time: 0.01s
Presolved: 1325 rows, 2450 columns, 7350 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 5.165165e+02, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  516.51648    0 

## Cardinality 3

In [9]:
s_3 = TspIntegerBCSolver(instance=i, cardinality=3)
solution = s_3.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 120150 rows, 2450 columns and 362600 nonzeros
Model fingerprint: 0x32aeddbf
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 2543.7515673
Presolve removed 79625 rows and 0 columns
Presolve time: 0.18s
Presolved: 40525 rows, 2450 columns, 124950 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)
Root relaxation presolved: 40525 rows, 2450 columns, 124950 nonzeros


Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 5.165165e+02, 109 iterations, 0.05 seconds (0.06 work units)

    Nodes    |    Current Node   